In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['data.h5', 'data.csv']


In [5]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# data.csv has 4009 rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('../input/data.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'data.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 4 columns



# Analysis 

- From a simple visual inspection it is possible to observe there is a perfect correlation between a set of trusted source and non fake news 


In [1]:
trusted_sites = ["www.bbc.com", "www.reuters.com", "www.nytimes.com", "www.abcnews.com", "cnn.com", "abcnews.go.com", "bbc.co.uk"]

In [6]:
def is_trusted(x): 
    for t in trusted_sites: 
        if t in x: 
            return 1 
    return 0 
        

# Feature Extraction 

- Mapping from Text URL to the corresponding Score and adding into a specific col 


In [7]:
df1["is_trusted_url"]=df1["URLs"].apply(is_trusted)

In [8]:
df1.head()

,URLs,Headline,Body,Label,is_trusted_url
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,1



# Feature Analysis  

- Check Correlation 


In [9]:
df1["Label"].corr(df1["is_trusted_url"])

1.0

Wow it's perfect :) 


# Solution Implementation 

- Let's build a simple mapping function using the URL feature 


In [10]:
def predict(x): 
    return is_trusted(x["URLs"])


# Test 

- Let's build a Test Set by random sampling 


In [12]:
n_test_set = 100
test_set = df1.sample(n=n_test_set)

# Let's pass all the possible features 
test_set_x = test_set[["URLs", "Headline", "Body"]]

# Label
test_set_y = test_set[["Label"]]

In [13]:
test_set_x.head()

,URLs,Headline,Body
525,https://www.reuters.com/article/us-trump-effec...,U.S. small-cap firms look to spend tax savings...,NEW YORK (Reuters) - A Texas-based chain of st...
734,http://www.bbc.com/news/world-us-canada-415738...,Can we tell if Donald Trump has a high IQ?,Image copyright AFP\nQuestion: How often does ...
166,https://www.activistpost.com/2017/10/catalan-i...,5 Things to Think About,By Tony Cartalucci\nHeadlines and commentary a...
402,http://beforeitsnews.com/sports/2017/09/bullpe...,"Bullpen Notes: September 25, 2017: Barraclough...","Bullpen Notes: September 25, 2017: Barraclough..."
501,http://beforeitsnews.com/u-s-politics/2017/09/...,Red Lobster Customer Makes Bomb Joke to Hostes...,Red Lobster Customer Makes Bomb Joke to Hostes...


In [14]:
test_set_y.head()

,Label
525,1
734,1
166,0
402,0
501,0


Let's predict the labels

In [15]:
# This passes to the predict function the columns (so axis=1) hence iterates over the rows (axis=0)
test_set_y_pred = test_set_x.apply(predict, axis=1)

In [16]:
test_set_y_pred.head()

525    1
734    1
166    0
402    0
501    0
dtype: int64

To compare the predicted and expected labels, let's just convert them into np arrays and subtract so any difference will result in a +1 or -1 hence false positive or negative 

In [17]:
res = test_set_y_pred.values.reshape(-1,1) - test_set_y.values.reshape(-1,1)

Let's look for false positive and negative 

In [30]:
# Any FP 
fp=res[res==+1].shape[0]
fn=res[res==-1].shape[0]
print(f"Num FP={fp}, Num FN={fn}")

Num FP=0, Num FN=0
